# Item-based Collaborative Filtering
* See `ItemCollaborativeFilteringBase.ipynb` for algorithm details
* The weights here are the cosine correlation between the two items

In [1]:
const name = "ExplicitItemCF";

In [2]:
import NBInclude: @nbinclude
@nbinclude("ExplicitItemCFBase.ipynb");

## Setup hyperparameters

In [3]:
item_cf_params = [
    cf_params(
        name = "ExplicitItemCF",
        training_residuals = ["ExplicitUserItemBiases"],
        validation_residuals = ["ExplicitUserItemBiases"],
        neighborhood_type = "abs",
        S = get_correlation_matrix_outdir(["ExplicitUserItemBiases"]),
        K = 1024,
        λ = [0.0, 1.0, 0.0, 0.0, 0.0],
    ),
];

## Train models

In [4]:
param = cf_params(
    name = "ExplicitItemCF",
    training_residuals = ["ExplicitUserItemBiases"],
    validation_residuals = ["ExplicitUserItemBiases"],
    neighborhood_type = "abs",
    S = get_correlation_matrix_outdir(["ExplicitUserItemBiases"]),
    K = 1024,
    λ = read_params("ExplicitItemCF")["λ"],
)

cf_params
  name: String "ExplicitItemCF"
  training_residuals: Array{String}((1,))
  validation_residuals: Array{String}((1,))
  neighborhood_type: String "abs"
  S: String "ExplicitItemCF/6540169344955467896"
  K: Int64 1024
  λ: Array{Float32}((5,)) Float32[0.0076675215, 3.8429337, 55.20206, 28.384584, 59.644646]


In [5]:
S = read_similarity_matrix(param.S)
K = param.K
neighborhood_types = Dict("abs" => get_abs_neighborhood)
neighborhoods = i -> neighborhood_types[param.neighborhood_type](i, S, K)

#84 (generic function with 1 method)

In [6]:
training = get_training(param.training_residuals)

Progress: 100%|███████████████████████████| Time: 0:00:00 (80.96 ns/it)


RatingsDataset
  user: Array{Int32}((224989805,)) Int32[1139733, 1139733, 1139733, 1139733, 1139733, 1139733, 1139733, 1139733, 1139733, 1139733  …  149272, 149272, 149272, 149272, 149272, 149272, 149272, 149272, 149272, 149272]
  item: Array{Int32}((224989805,)) Int32[18692, 12822, 14577, 14269, 6834, 12622, 12776, 11770, 9027, 9564  …  378, 12008, 8924, 20136, 19379, 7835, 16002, 2085, 12932, 2415]
  rating: Array{Float32}((224989805,)) Float32[-0.39215803, 0.24017143, 1.1383104, 0.16493988, 1.2026196, 2.5712285, -2.8432922, -1.8642883, 1.979228, 0.17419434  …  -0.9521179, 0.9328785, 1.3864756, -1.1284418, -0.3624091, -1.7360964, 0.5852394, -0.21881008, 0.6126938, 0.35669422]


In [7]:
function model(users, items)
    inference = RatingsDataset(users, items, fill(NaN32, length(users)))
    collaborative_filtering(training, inference, neighborhoods, param.λ)
end;

In [ ]:
write_alpha(model, param.validation_residuals, false, param.name; log_splits=false)

Progress:  10%|██▊                        |  ETA: 6:14:33 (10.10  s/it)mt)m